In [1]:
# check copyright of the game (you bought the game, right?)
urbek_data = r"H:\Steam\steamapps\common\Urbek\Urbek_Data"
with open(fr"{urbek_data}\app.info") as f:
    print(f.read())

Estudios Kremlinois
Urbek


In [11]:
import os
import re
import json
import pandas
import UnityPy
import numpy as np
from collections import defaultdict

pandas.set_option('display.max_rows', 1000)
pandas.set_option('display.max_columns', None)
pandas.set_option('display.max_colwidth', None)

<module 're' from 'C:\\ProgramData\\Anaconda3\\lib\\re.py'>

In [3]:
# load data file from game installation
env = UnityPy.load(fr"{urbek_data}\resources.assets")

rules = {}
translations = {}
for obj in env.objects:
    if obj.type.name != "TextAsset": continue
    data = obj.read()
    # download json files for browsing:
    os.makedirs("./data", exist_ok=True)
    with open(f"./data/{data.name}.json", "wb") as f:
        f.write(data.script)
    if data.name == "gameValues":
        for item in json.loads(data.text):
            translations[item["id"]] = item
    if re.match(r"reglasBioma_\d$", data.name) and \
       re.search(r"localVariables", data.text):
        rules[data.name] = json.loads(data.text)
del env, obj, data

# explore list of languages
translations["resources_comida"]

{'id': 'resources_comida',
 'INFO': 'name of the resources',
 'en': 'Food',
 'es': 'Comida',
 'fr': 'Nourriture',
 'zh-CN': '食物',
 'pl': 'Jedzenie ',
 'de': 'Nahrungsmittel  ',
 'pt': 'Comida',
 'ru': 'Еда',
 'ja': '食品',
 'it': 'Alimentazione',
 'ko': '음식'}

In [4]:
# choose a language
language = "en"
labels = dict((k, v[language])
              for k, v in translations.items()
              if not re.search(r"text|desc|name|effect|t_|nr_", k))

# explore list of (non-empty) biomes
dict(sorted([(k, re.sub(r"<b>|</b>[\s\S]*", "", v["description"])) for k, v in rules.items()]))

{'reglasBioma_0': 'Temperate',
 'reglasBioma_1': 'Desert',
 'reglasBioma_2': 'Archipelago',
 'reglasBioma_3': 'Forest',
 'reglasBioma_4': 'Ruins'}

In [28]:
# choose a biome
biome = "reglasBioma_4"
data = rules[biome]
columns_resources = [item.split(',')[0] for item in data["resources"]]
columns_lv = [item.split(',')[0] for item in data["localVariables"]]

# parse buildings form the biome
buildings = []
updatesFromDict = defaultdict(list)
for building in data["construs"]:
    if up := building.get('updates'):
        for u in up:
            updatesFromDict[u].append(building['codeName'])
    
for building in sorted(data["construs"], key=lambda b: next((x['q'] for x in b.get('localVariables', []) if x['locVar'] == 'densidad'), 999)):
    codeName = building['codeName']
    label = labels.get(f'b_{codeName}', '???')
    category = building.get('category')
    category = f"{category} ({labels.get(f'bg_{category}')})" if category else ""
    distanceToRoad = building.get('distanceToRoad', '')
    updates = building.get('updates', '')
    updatesFrom = updatesFromDict.get(codeName, '')
    buildingsNeeded = [f"{'≤' if not p['biggerThan'] else ''}{p['q']} {p['building']} ⊙{p['radio'] if p['radio'] else '∞'}"
                       for p in building.get('buildingsNeeded')
                      ] if building.get('buildingsNeeded') else ''
    unblock = [f"{p['q']} {p.get('gloVar', p.get('nBuildings', '?'))}"
               for p in building.get('unblock')
              ] if building.get('unblock') else ''
    minModels = min([re.sub(r"^[^,]*,|,E?$", "", p) for p in building.get('models') if p
                    ]) if building.get('models') else ''
    
    lvn = [next((f"{'≤' if not p['biggerThan'] else ''}{str(p['q']) + '□ ' + str(p['threshold']).rjust(2) if 'threshold' in p else p['q']} ⊙{p['radio']}"
                 for p in building.get('localVariablesNeeded', [])
                 if p['localVariable'] == col), "")
           for col in columns_lv]
    lv = [next((f"{p['q']:+}"
                 for p in building.get('localVariables', [])
                 if p['locVar'] == col), "")
          for col in columns_lv]
    rn = [next((f"{'≤' if not p['biggerThan'] else ''}{p['q']} ⊙{p['radio']}"
                for p in building.get('resourcesNeeded', [])
                if p['resource'] == col), "")
          for col in columns_resources]
    rx = [next((f"-{p['q']} ⊙{p['distance']}"
                for p in ([building.get('resourceExtraction')] if building.get('resourceExtraction') else [])
                if p['resource'] == col), "")
          for col in columns_resources]
    r = [next((f"{p['q']:+}"
               for p in building.get('produces', [])
               if p['rec'] == col), "")
         for col in columns_resources]
    
    lvnlv = [f"{a}{' '+b.rjust(3) if b else ''}" for a, b in zip(lvn, lv)]
    rnrxr = [f"{a}{' '+b.rjust(3) if b else ''}{' '+c.rjust(3) if c else ''}" for a, b, c in zip(rn, rx, r)]
    
    if any([updates, unblock, *rn, *rx, *r, *lvn, *lv]):
        buildings.append([label,
                          distanceToRoad,
                          minModels,
                          *lvnlv,
                          *rnrxr,
                          codeName,
                          buildingsNeeded,
                          unblock,
                          updatesFrom,
                          updates,
                          category,
                         ])

# visualize buildings
columns = [f"Building name ({language})", "distanceToRoad", "smallest models",
           *[f"{labels[f'lv_{c}']}  ({c})" for c in columns_lv],
           *[f"{labels[f'resources_{c}']}  ({c})" for c in columns_resources],
           "codeName", "buildingsNeeded", "unblock", "updatesFrom", "updates", "category",
          ]
df = pandas.DataFrame(buildings, columns=columns)
# df.sort_values([("produces", "Science  (ciencia)")], key=pandas.to_numeric, ascending=False)
# df

In [29]:
df

,Building name (en),distanceToRoad,smallest models,residents (densidad),leisure (ocio),happiness (comodidad),education (educacion),food services (servicios),tools (herramientas),sports (deporte),retail (comercio),offices (oficinas),subway station (lineaMetro),logistics (logistica),green areas (areasVerdes),night life (vidaNocturna),connectivity (conectividad),self-organization (autoorganizacion),culture (cultura),faith (fe),pollution (contaminacion),iron industry (produccionHierro),timber industry (produccionMadera),healthcare (salud),banking (banca),scavenging food (saqueoComida),scavenging iron (saqueoHierro),Food (comida),Work (trabajo0),Skilled work (trabajo1),High skilled work (trabajo2),Wood (madera),Iron (hierro),Coal (carbon),Energy (energia),Steel (acero),Goods (bienes),Science (ciencia),Biomass (biomasa),Radioactivity (radioactividad),codeName,buildingsNeeded,unblock,updatesFrom,updates,category
0,Haunted house,3,"2,2",+1,,+20,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,fantasma,[4 cementerio ⊙3],"[10 bienestar, 6500 poblacion]",[villa2],,ciudad (Landmark)
1,Survivalist house,3,"2,2",+4,,+4,,,,,,,,,,,,,,,,,,,,4 ⊙4,,,+4,,,,,,,,,,,,sobreviviente,,,,,casas (Houses)
2,Wooden hut,3,"1,1",+6,,+6,,,,,,,,,,,,,,,,,,,,,,-10,+10,,,,,,,,,,,,cabanha,,,"[protoCabanha, cabanha, puebloPobre, pueblo, villa, fosforo, poblaPobre, pobla, ciudad, ciudad2, moderna, mixta, mixta5, mixta8, edificio5verde, mixta5verde, callampa, bloques, pechonhos, edificio5, edificio12]","[ciudad2, ciudad, lota, lotaCarbon, fosforo, villa2, villa, pueblo, puebloPobre, callampa, cabanha]",
3,Villa,3,"2,2",≤25 ⊙4 +6,,+13,,,,,,,,,1 ⊙4,,,,,,≤0 ⊙5,,,,,,,-22,+10,,,,,,-2,,,,,,villa,,[800 poblacion],"[cabanha, puebloPobre, pueblo, villa, fosforo, poblaPobre, pobla, ciudad, ciudad2, moderna, mixta, mixta5, mixta8, edificio5verde, mixta5verde, pechonhos, edificio5, edificio12]","[villa2, villa, pueblo, puebloPobre, callampa, cabanha]",
4,Village house,3,"1,1",15□ 6 ⊙3 +8,,+8,,,,,,,,,,,,,,,≤0 ⊙5,,,,,,,-12,+14,,,,,,-1,,,,,,puebloPobre,,[200 poblacion],"[protoCabanha, cabanha, puebloPobre, pueblo, villa, fosforo, poblaPobre, pobla, ciudad, ciudad2, moderna, mixta, mixta5, mixta8, edificio5verde, mixta5verde, callampa, pechonhos, edificio5, edificio12]","[pobla, poblaPobre, villa, pueblo, puebloPobre, callampa, cabanha]",
5,Wealthy village house,3,"1,1",15□ 6 ⊙3 +8,1 ⊙5,+11,,1 ⊙5,,,,,,,,,,,,,≤0 ⊙5,,,,,,,-18,+14,,,,,,-2,,,,,,pueblo,,[1300 poblacion],"[protoCabanha, cabanha, puebloPobre, pueblo, villa, fosforo, poblaPobre, pobla, ciudad, ciudad2, moderna, mixta, mixta5, mixta8, edificio5verde, mixta5verde, pechonhos, edificio5, edificio12]","[pobla, poblaPobre, villa, pueblo, puebloPobre, callampa, cabanha]",
6,Poor suburbs,3,"1,1",+10,,+7,,,,,,,,,,,,,,,≤0 ⊙5,,,,,,,-10,+14,,,,,,-1,,,,,,fosforo,[1 plazaCondominio ⊙3],"[2200 poblacion, 1 escuela]","[protoCabanha, cabanha, fosforo, pechonhos]","[pechonhos, fosforo, pobla, poblaPobre, villa, pueblo, puebloPobre, callampa, cabanha]",
7,Suburb house,3,"1,1",15□ 8 ⊙3 +10,1 ⊙6,+10,1 ⊙8,1 ⊙6,,,,,,,,,,,,,≤0 ⊙5,,,,,,,-16,+14,+2,,,,,-3,,,,,,poblaPobre,,"[1800 poblacion, 1 escuela]","[puebloPobre, pueblo, fosforo, poblaPobre, pobla, ciudad, ciudad2, moderna, mixta, mixta5, mixta8, edificio5verde, mixta5verde, estacionamiento, pechonhos, edificio5, edificio12]","[pechonhos, ciudad, estacionamiento, pobla, poblaPobre, villa, pueblo, puebloPobre, callampa, cabanha]",
8,Wealthy suburb house,3,"1,1",15□ 8 ⊙3 +10,3 ⊙6,+13,1 ⊙8,1 ⊙6,,,,,,,,,,,,,≤0 ⊙5,,,,,,,-22,+12,+3,,,,,-4,,,,,,pobla,,[4100 poblacion],"[puebloPobre, pueblo, fosforo, poblaPobre, pobla, ciudad, ciudad2, moderna, mixta, mixta5, mixta8, edificio5verde, mixta5verde, estacionamiento, pechonhos, edificio5, edificio12]","[pechonhos, ciudad, estacionamiento, pobla, poblaPobre, villa, pueblo, puebloPobre, callampa, cabanha]",
9,Iron miners' house,1,"1,1",+10,,+6,,,,,,,,,,,,,,,,1 ⊙5,,,,,,-10,,,,,+1,,-1,,,,,,lota,,"[1700 poblacion, 1 minaHierro]","[protoCabanha, cabanha, lota, lota3]","[lota3, lo

In [6]:
%%html
<style>
  #notebook-container {
      width: 99%;
  }
  body,
  div#notebook,
  div.output_subarea {
      overflow: visible; /* for the `position: sticky` */
  }
  .dataframe {
      position: relative;
  }
  .dataframe thead {
      position: sticky;
      top: 0;
      background: black;
      z-index: 1;
  }
  .dataframe thead tr {
      height: 250px;
  }
  .dataframe thead th {
      white-space: pre;
      vertical-align: bottom;
      text-align: left !important;
  }
  .dataframe thead tr th:nth-child(n+3):not(:nth-last-child(-n+6)) {
      transform-origin: bottom left;
      transform: translateX(20px) rotate(-60deg);
      max-width: 20px;
  }
  .dataframe tr:hover {
      background: black !important;
  }
  .dataframe tr:hover {
      background: black !important;
  }
  .dataframe td {
      white-space: pre;
      border-left: 1px dotted #333;
  }
  .dataframe td:nth-child(2) {
      position: sticky;
      left: 0;
      background: inherit;
  }
  .dataframe td:nth-child(n+3) {
      text-align: right;
      font-family: monospace !important;
  }
  .dataframe td:nth-last-child(-n+6) {
      text-align: left;
      max-width: 250px;
      overflow: hidden;
      white-space: nowrap;
      text-overflow: ellipsis;
      line-height: 1.2;
  }
  .dataframe td:nth-last-child(-n+5):not(:last-child):hover {
      position: absolute;
      white-space: pre-wrap;
      background: black;
  }
</style>